In [71]:
import os
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser

In [ ]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "XX"
os.environ["LANGSMITH_ENDPOINT"] = "https://eu.api.smith.langchain.com"

In [73]:
# For better security, load environment variables from a .env file
# from dotenv import load_dotenv
# load_dotenv()
# Make sure your OPENAI_API_KEY is set in the .env file

# Initialize the Language Model (using ChatOpenAI is recommended)
llm = ChatOllama(base_url="127.0.0.1:11434/engines/v1"
                              ,model="qwen3:0.6b"
                              ,api_key="ignored"
                              ,temperature=0)

# Define the structured output schema for the transformation step
class LaptopSpecs(BaseModel):
    cpu: str = Field(description="CPU model or speed")
    memory: str = Field(description="RAM amount")
    storage: str = Field(description="Storage size and type")

# Wrap the LLM to return structured output that matches the schema
structured_llm = llm.with_structured_output(LaptopSpecs)

In [74]:
# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_template(
    "Extract the technical specifications from the following text:\n\n{text_input}"
)

In [75]:
# --- Prompt 2: Transform to Structured Output ---
prompt_transform = ChatPromptTemplate.from_template(
    "Transform the following specifications into structured output with 'cpu', 'memory', and 'storage' as fields:\n\n{specifications}"
)

In [76]:
# --- Build the Chain using LCEL ---
# The StrOutputParser() converts the LLM's message output to a simple string.
extraction_chain = prompt_extract | llm | StrOutputParser()

In [77]:
# The full chain passes the output of the extraction chain into the 'specifications'
# variable for the transformation prompt.
full_chain = (
    {"specifications": extraction_chain}
    | prompt_transform
    | structured_llm
)

In [78]:
# --- Run the Chain ---
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."

# Execute the chain with the input text dictionary.
final_result = full_chain.invoke({"text_input": input_text})

print("\n--- Final Structured Output ---")
print(final_result)

# Pydantic v2 uses model_dump; v1 uses dict.
result_dict = final_result.model_dump()
print("\n--- As Dict ---")
print(result_dict)


--- Final Structured Output ---
cpu='3.5 GHz octa-core' memory='16GB' storage='1TB NVMe SSD'

--- As Dict ---
{'cpu': '3.5 GHz octa-core', 'memory': '16GB', 'storage': '1TB NVMe SSD'}
